<a href="https://colab.research.google.com/github/AndreLobo1/Aprimoramento/blob/main/limpeza_tabela_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Carregar a biblioteca pandas

In [ ]:
import pandas as pd

Importa o arquivo

In [ ]:
file_path = '/content/drive/MyDrive/AUDIENCIA_07_08_2024_COMPLETA.csv'
df = pd.read_csv(file_path)

Verificação de missing values: Não foram encontrados missing values na tabela, portanto, não é necessário realizar nenhum tratamento adicional.








In [ ]:
missing_values = df.isnull().sum()
print("Missing values por coluna:")
print(missing_values[missing_values > 0])

Criar um dicionário para mapear os meses em português para números


In [ ]:
meses = {
    'jan.': 1, 'fev.': 2, 'mar.': 3, 'abr.': 4,
    'mai.': 5, 'jun.': 6, 'jul.': 7, 'ago.': 8,
    'set.': 9, 'out.': 10, 'nov.': 11, 'dez.': 12
}

Separar a coluna "PERIODO" em "ANO" e "MÊS"*


In [ ]:
df['ANO'] = df['PERIODO'].apply(lambda x: '20' + x.split('-')[1])
df['MÊS'] = df['PERIODO'].apply(lambda x: meses[x.split('-')[0]])

Elimina a coluna 'PERIODO'

In [ ]:
df = df.drop(columns=['PERIODO'])

Reordenar as colunas para que "ANO" e "MÊS" sejam as primeiras

In [ ]:
colunas = ['ANO', 'MÊS'] + [col for col in df.columns if col not in ['ANO', 'MÊS']]
df = df[colunas]

Verificar tipos de dados das colunas e se existem colunas que precisam ser convertidas


In [ ]:
print("Tipos de dados antes da conversão:")
print(df.dtypes)

colunas_para_converter = df.select_dtypes(include=['object']).columns
print("Colunas para converter para numérico:", list(colunas_para_converter))

As colunas numéricas são:

- MÊS
- TV GAZETA
- GAZETA FM VITÓRIA   
- LITORAL FM          
- CBN VITORIA         
- RÁDIO MIX VITÓRIA  

Colunas que necessitam de tratamento para numérico:

- ANO
- INTERNET
- PORTAL G1/GE/HOME

Converter colunas 'INTERNET' e 'PORTAL G1/GE/HOME' para numérico: Função para remover separadores de milhar e converter para numérico


In [ ]:
def converter_para_numerico(valor):
    if isinstance(valor, str):
        valor = valor.replace('.', '')
        valor = valor.replace(',', '.')
        return pd.to_numeric(valor, errors='coerce')
    return valor

Aplicar a função de conversão nas colunas "INTERNET" e "PORTAL G1/GE/HOME"


In [ ]:
df['INTERNET'] = df['INTERNET'].apply(converter_para_numerico)
df['PORTAL G1/GE/HOME'] = df['PORTAL G1/GE/HOME'].apply(converter_para_numerico)

Converter a coluna "ANO" para numérico: Agora todas as colunas são numéricas.

In [ ]:
df['ANO'] = pd.to_numeric(df['ANO'], errors='coerce')

Identificar colunas categóricas e numéricas

In [ ]:
colunas_categoricas = df.select_dtypes(include='object').columns
colunas_numericas = df.drop(colunas_categoricas, axis=1).columns

print(f'Há {len(colunas_categoricas)} Colunas Categóricas: {list(colunas_categoricas)}')
print(f'Há {len(colunas_numericas)} Colunas Numéricas: {list(colunas_numericas)}')


Identificar outliers: Os outliers identificados estão corretos, não sendo necessário nenhum tratamento adicional

In [ ]:
Q1 = df[colunas_numericas].quantile(0.25)
Q3 = df[colunas_numericas].quantile(0.75)
IQR = Q3 - Q1

outliers = ((df[colunas_numericas] < (Q1 - 1.5 * IQR)) | (df[colunas_numericas] > (Q3 + 1.5 * IQR)))

outlier_counts = outliers.sum()
print("Outliers por coluna:")
print(outlier_counts[outlier_counts > 0])

for coluna in outlier_counts[outlier_counts > 0].index:
    print(f"\nOutliers na coluna '{coluna}':")
    print(df.loc[outliers[coluna], coluna])

Arredondar os valores numéricos para 3 casas decimais


In [ ]:
df[colunas_numericas] = df[colunas_numericas].round(3)

Exibir as primeiras linhas para verificar a limpeza

In [ ]:
print("Primeiras linhas após a reordenação das colunas:")
print(df.head())